In [ ]:
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h516909a_0.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py37ha21ca33_2.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.7/dist-packages/ lib/python3.7/site-packages/talib --strip-components=3

In [ ]:
!pip install python-binance
!pip install binance-client

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [2]:
from get_data import crypto_data
from get_data import prepare_data

In [3]:
timeperiods = {}
timeperiods["BB"] = 700
timeperiods["EMA"] = 700
timeperiods["ADX0"] = 700
timeperiods["ADX0var"] = 700
timeperiods["ATR0"] = 700
timeperiods["ARO0"] = 700
timeperiods["WLR0"] = 700

In [ ]:
df = crypto_data(timeperiods)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow import keras

In [ ]:
#test method
def test_params(timeperiods):
  df = crypto_data(timeperiods)
  print("======================================================")
  print("test for periods: ", timeperiods)
  example = df["df_BTC"]
  df_input = pd.DataFrame(columns = example.columns)
  for single in df.values():
    df_input = df_input.append(single)

  df_output = np.zeros(df_input.shape[0])
  for i in range(1, df_input.price.shape[0]):
    diff = df_input.price.iloc[i] - df_input.price.iloc[i-1]
    if diff > 0:
      df_output[i] = 1
    elif diff < 0:
      df_output[i] = -1

  df_input_test = df_input.iloc[::5]
  df_input_train = df_input.iloc[1::5]
  df_input_train = df_input_train.append(df_input.iloc[2::5])
  df_input_train = df_input_train.append(df_input.iloc[3::5])
  df_input_train = df_input_train.append(df_input.iloc[4::5])

  df_output_test = df_output[::5]
  df_output_train = df_output[1::5]
  df_output_train = np.concatenate((df_output_train, df_output[2::5]))
  df_output_train = np.concatenate((df_output_train, df_output[3::5]))
  df_output_train = np.concatenate((df_output_train, df_output[4::5]))


  clf = tree.DecisionTreeClassifier(max_depth=7)
  clf = clf.fit(df_input_train, df_output_train)
  preds = clf.predict(df_input_test)
  print("decision tree score test: ", 
        clf.score(df_input_test, df_output_test), 
        " up preds: ", np.sum(preds == 1),
        " down preds: ", np.sum(preds == -1),
        " 0 preds: ", np.sum(preds == -1))
  print("decision tree score train: ", clf.score(df_input_train, df_output_train))


  gnb = GaussianNB()
  gnb.fit(df_input_train, df_output_train)
  preds = gnb.predict(df_input_test)
  print("Gaussian NB score test: ", 
        gnb.score(df_input_test, df_output_test),
        " up preds: ", np.sum(preds == 1),
        " down preds: ", np.sum(preds == -1),
        " 0 preds: ", np.sum(preds == -1)
  )
  print("Gaussian NB score train: ", gnb.score(df_input_train, df_output_train))

  gnb = MultinomialNB()
  gnb.fit(df_input_train, df_output_train)
  preds = gnb.predict(df_input_test)
  print("Multinomial NB score test: ", 
        gnb.score(df_input_test, df_output_test),
        " up preds: ", np.sum(preds == 1),
        " down preds: ", np.sum(preds == -1),
        " 0 preds: ", np.sum(preds == -1)
  )
  print("Multinomial NB score train: ", gnb.score(df_input_train, df_output_train))



  clf = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0)
  clf.fit(df_input_train, df_output_train)
  preds = clf.predict(df_input_test)
  print("random forest score test: ", 
        clf.score(df_input_test, df_output_test),
        " up preds: ", np.sum(preds == 1),
        " down preds: ", np.sum(preds == -1),
        " 0 preds: ", np.sum(preds == -1)
  )
  print("random forest score train: ", clf.score(df_input_train, df_output_train))



  #model preparation
  ds_output_NN_train = tf.one_hot(df_output_train, depth = 3)
  ds_output_NN_test = tf.one_hot(df_output_test, depth = 3)

  model = tf.keras.models.Sequential([                    
    #tf.keras.layers.Conv1(filters= 2, kernel_size = 3, activation='tanh', use_bias=False),
    #tf.keras.layers.MaxPool2D(),
    #tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, use_bias=True, activation="relu", input_shape=(df_input.shape[1],)),
    tf.keras.layers.Dense(5, use_bias=True, activation="tanh"),
    tf.keras.layers.Dense(3, use_bias=True, activation="softmax")
  ])

  model.compile(
      optimizer=tf.keras.optimizers.Adam(0.0008),
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[tf.keras.metrics.CategoricalAccuracy()],
  )

  model.fit(
      df_input_train,
      ds_output_NN_train,
      epochs=10,
      verbose = 0
  )

  print("NN score: ")
  model.evaluate(df_input_test, ds_output_NN_test, verbose=1)
  preds = model.predict(df_input_test)
  print("NN score test: ", 
        " up preds: ", np.sum(preds == 1),
        " down preds: ", np.sum(preds == -1),
        " 0 preds: ", np.sum(preds == -1)
  )
  print("======================================================")

In [ ]:
for timeperiod  in [100, 200, 500, 700, 750, 800, 900, 1200]:
  timeperiods = {}
  timeperiods["BB"] = timeperiod
  timeperiods["EMA"] = timeperiod
  timeperiods["ADX0"] = timeperiod
  timeperiods["ADX0var"] = timeperiod
  timeperiods["ATR0"] = timeperiod
  timeperiods["ARO0"] = timeperiod
  timeperiods["WLR0"] = timeperiod
  test_params(timeperiods)